In [ ]:
# Import
from pyomo.environ import *

# Creation of a Concrete Model
model = ConcreteModel()

# Sets
model.i = Set(initialize=['gen1','gen2', 'gen3','gen4'], doc='generadores')
model.j = Set(initialize=['per1','per2','per3'], doc='periodos')
model.s = Set(initialize=['s1','s2','s3'], doc='escenarios de demanda')


# Parameters
model.F = Param(model.i, initialize = {'gen1':10, 'gen2':7, 'gen3':16, 'gen4':6}, doc = 'coste fijo de inversion')
model.PROBS = Param(model.s, initialize={'s1':0.2,'s2':0.5,'s3':0.3}, doc='probabilidad de cada escenario [p.u.]')


# Tables
table1 = {
    ('gen1',  'per1') : 40,
    ('gen1',  'per2') : 24,
    ('gen1',  'per3') :  4,
    ('gen2',  'per1') : 45,
    ('gen2',  'per2') : 27,
    ('gen2',  'per3') : 4.5,
    ('gen3',  'per1') : 32,
    ('gen3',  'per2') : 19.2,
    ('gen3',  'per3') : 3.2,
    ('gen4',  'per1') : 55,
    ('gen4',  'per2') : 33,
    ('gen4',  'per3') : 5.5,
}


model.V = Param(model.i, model.j, initialize=table1, doc ='coste variable de operación [€ por MW]')


table2 = {
    ('s1','per1') : 3,
    ('s1','per2') : 3,
    ('s1','per3') : 2,
    ('s2','per1') : 5,
    ('s2','per2') : 3,
    ('s2','per3') : 2,
    ('s3','per1') : 7,
    ('s3','per2') : 3,
    ('s3','per3') : 2,
}


model.DEMS = Param(model.s, model.j, initialize=table2, doc ='demanda estocástica [MW]')


def dem_c(model,j):
    return sum(model.DEMS[s,j] * model.PROBS[s] for s in model.s) 
model.DEM = Param(model.j, initialize=dem_c, doc='demanda para un escenario [MW]')



# Scalar
model.POTMIN = Param(initialize=12, doc='potencia mínima a instalar [MW]')
model.PRSPTO = Param(initialize=120, doc='límite presupuestario [€]')


# Variables
model.X = Var(model.i, bounds=(0.0,None), doc='potencia a instalar [MW]')
model.Y = Var(model.j, model.i, model.s, bounds=(0.0,None), doc='potencia en operación [MW]')

# Equations
def coste_rule(model):
    return  sum(model.F[i] * model.X[i]  for i in model.i) + sum(model.PROBS[s] * model.V[i,j] * model.Y[j,i,s] for i in model.i for j in model.j for s in model.s)
model.coste = Objective(rule=coste_rule, sense=minimize, doc='coste total [€]')


def  presup_rule(model):
    return sum(model.F[i] * model.X[i] for i in model.i ) == model.PRSPTO
model.presup = Constraint(rule=presup_rule, doc='limitación presupuestaria [€]')


def  insmin_rule(model):
    return sum(model.X[i] for i in model.i ) >= model.POTMIN
model.insmin = Constraint(rule=insmin_rule, doc='potencia mínima instalada [MW]')


def  balpot_rule(model,j,i,s):
    return model.Y[j,i,s] <= model.X[i]
model.balpot = Constraint(model.j, model.i, model.s, rule=balpot_rule, doc='potencia en operación menor que instalada [MW]')


#BALDEM(j) .. sum(i, Y(j,i)) =G= DEM(j)

def  baldem_rule(model,j,s):
    return sum(model.Y[j,i,s] for i in model.i) >= model.DEMS[j,s] ; 
model.baldem = Constraint(model.j,model.s, rule=baldem_rule, doc='balance de demanda [MW]')


# SOLVE 
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
results = opt.solve(model)
 #sends results to stdout
results.write()

# display solution
model.display()